# AutoBuild
AutoGen ofrece agentes conversables impulsados por LLM, herramienta o humano, que se pueden utilizar para realizar tareas colectivamente a través de chat automatizado. Este marco permite el uso de herramientas y la participación humana a través de conversación multiagente.
Por favor, encuentre la documentación sobre esta característica [aquí](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

En este cuaderno, presentamos una nueva clase, `AgentBuilder`, para ayudar al usuario a construir un proceso de resolución de tareas automático impulsado por un sistema multiagente. Específicamente, en `build()`, solicitamos a un LLM que cree varios agentes participantes e inicialice un chat grupal, y especificamos si esta tarea necesita programación para resolverse. AgentBuilder también soporta LLMs de código abierto por [vLLM](https://docs.vllm.ai/en/latest/index.html) y [Fastchat](https://github.com/lm-sys/FastChat). Consulte la lista de modelos soportados [aquí](https://docs.vllm.ai/en/latest/models/supported_models.html).

## Requisito

AutoBuild necesita la última versión de AutoGen.
Puede instalar AutoGen con el siguiente comando:

In [ ]:
%pip install --upgrade pyautogen

## Paso 1: preparar la configuración
Prepare un `config_path` para el agente asistente para limitar la elección del LLM que desea utilizar en esta tarea. Esta configuración puede ser una ruta de archivo json o un nombre de variable de entorno. También se requiere un `default_llm_config` para inicializar la configuración específica de los LLMs como semilla, temperatura, etc...

In [ ]:
config_path = 'OAI_CONFIG_LIST.json'  # modify path
default_llm_config = {
    'temperature': 0
}

## Paso 2: crear un AgentBuilder
Cree un `AgentBuilder` con el `config_path` especificado. Por defecto, AgentBuilder utilizará GPT-4 para completar todo el proceso, también puede cambiar el `builder_model` a otro modelo de OpenAI si lo desea. También puede especificar un LLM de OpenAI o de código abierto como columna vertebral del agente, vea el blog para más detalles.

In [ ]:
from autogen.agentchat.contrib.agent_builder import AgentBuilder

builder = AgentBuilder(config_path=config_path, builder_model='gpt-4-1106-preview', agent_model='gpt-4-1106-preview')

## Paso 3: especificar una tarea de construcción

Especifique una tarea de construcción con una descripción general. La tarea de construcción ayudará al gestor de construcción (un LLM) a decidir qué agentes se deben construir.

In [ ]:
building_task = "Corregir, mejorar y traducir textos del español al inglés y viceversa. Cuando el texto este terminado salvar un markdown con toda la informacion y presentado de una forma correcta y legible."

## Paso 4: construir agentes de chat grupal
Use `build()` para permitir que el gestor de construcción (el `builder_model` especificado) complete la generación de agentes de chat grupal. Si cree que la codificación es necesaria en su tarea, puede usar `coding=True` para agregar un proxy de usuario (un intérprete de código automático) a la lista de agentes, como:
```python
builder.build(building_task, default_llm_config, coding=True)
```
Si no se especifica `coding`, AgentBuilder determinará por sí mismo si se debe agregar o no el proxy de usuario de acuerdo con la tarea.

In [ ]:
agent_list, agent_configs = builder.build(building_task, default_llm_config)

## Paso 5: ejecutar la tarea
Permita que los agentes generados en `build()` completen la tarea de manera colaborativa en un chat grupal.

In [ ]:
import autogen

def start_task(execution_task: str, agent_list: list, llm_config: dict):
    config_list = autogen.config_list_from_json(config_path, filter_dict={"model": ["gpt-4-1106-preview"]})
    
    group_chat = autogen.GroupChat(agents=agent_list, messages=[], max_round=12)
    manager = autogen.GroupChatManager(
        groupchat=group_chat, llm_config={"config_list": config_list, **llm_config}
    )
    agent_list[0].initiate_chat(manager, message=execution_task)

start_task(
    # execution_task es la tarea que el equipo de agentes debe realizar
    execution_task="Crea un texto dando la vienvenida a los nuevos estudiantes de mi academia de gimnasia artistica en español y en ingles.",
    agent_list=agent_list,
    llm_config=default_llm_config
)

## Guardar y cargar configuraciones

Puedes guardar toda la información necesaria de los agentes de chat grupal construidos. Aquí hay un caso para esos agentes generados en la tarea anterior:

Esta información se guardará en formato JSON. Puedes proporcionar un nombre de archivo específico, de lo contrario, AgentBuilder guardará la configuración en la ruta actual con un nombre de archivo generado 'save_config_TASK_MD5.json'.

In [ ]:
saved_path = builder.save()

Después de eso, puedes cargar la configuración guardada y saltarte el proceso de construcción. AgentBuilder creará agentes con esa información sin solicitar al gestor de construcción.

In [ ]:
new_builder = AgentBuilder(config_path=config_path)
agent_list, agent_configs = new_builder.load(saved_path)  # load previous agent configs
start_task(
    execution_task="Corregir, mejorar y traducir textos del español al inglés y viceversa. Cuando el texto este terminado salvar un markdown con toda la informacion y presentado de una forma correcta y legible.",
    agent_list=agent_list,
    llm_config=default_llm_config
)
new_builder.clear_all_agents()

## (Opcional): limpiar todos los agentes y prepararse para la próxima tarea
Puedes limpiar todos los agentes generados en esta tarea con el siguiente código si tu tarea se ha completado o si la próxima tarea es muy diferente de la tarea actual. Si la columna vertebral del agente es un LLM de código abierto, este proceso también cerrará el servidor de punto final. Si es necesario, puedes usar `recycle_endpoint=False` para mantener el servidor de punto final de los LLMs de código abierto anteriores.

In [ ]:
builder.clear_all_agents(recycle_endpoint=True)

## Utilizar el Asistente de OpenAI

[La API de Asistentes](https://platform.openai.com/docs/assistants/overview) te permite construir asistentes de IA dentro de tus propias aplicaciones. Un Asistente tiene instrucciones y puede aprovechar modelos, herramientas y conocimientos para responder a las consultas de los usuarios.
AutoBuild también soporta la API de asistentes añadiendo `use_oai_assistant=True` a `build()`.

In [ ]:
new_builder = AgentBuilder(config_path=config_path)
agent_list, agent_configs = new_builder.build(building_task, default_llm_config, use_oai_assistant=True)  # Transfer to OpenAI assistant API.
start_task(
    execution_task="Find a recent paper about XAI on arxiv and find its potential applications in medical.",
    agent_list=agent_list,
    llm_config=default_llm_config
)
new_builder.clear_all_agents()